In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from lib.zigzag import zig_zag
import lib.bars as bars

In [10]:
data = pd.read_json("..\Datasets\WDO@_M5.json")
columns = ['time', 'open', 'high', 'low', 'close']
# columns = ['time', 'open', 'high', 'low', 'close', 'zz', 'highs', 'lows']
data = data.loc[:, columns]
data['time']=pd.to_datetime(data['time'], unit='ms')
data.tail()

,time,open,high,low,close
96519,2021-04-05 17:35:00,5677.5,5685.0,5677.5,5685.0
96520,2021-04-05 17:40:00,5685.0,5685.5,5682.5,5684.0
96521,2021-04-05 17:45:00,5684.5,5686.0,5676.5,5679.5
96522,2021-04-05 17:50:00,5679.0,5680.5,5676.0,5677.0
96523,2021-04-05 17:55:00,5676.5,5679.0,5672.0,5672.0


In [3]:
# parsed_ds=data.loc[(data['time']>='2021-03-19 8:00:00')&(data['time']<='2021-03-19 19:00:00')]
parsed_ds=data.loc[(data['time']>='2021-03-19 09:00:00')&(data['time']<='2021-03-22 19:00:00')]
parsed_ds.reset_index(inplace=True, drop=True)
parsed_ds.head()

,time,open,high,low,close
0,2021-03-19 09:00:00,5530.0,5541.5,5525.5,5533.5
1,2021-03-19 09:05:00,5533.0,5541.0,5525.5,5527.5
2,2021-03-19 09:10:00,5527.5,5540.0,5526.0,5538.5
3,2021-03-19 09:15:00,5538.5,5546.0,5530.5,5545.0
4,2021-03-19 09:20:00,5545.0,5553.0,5543.0,5548.0


In [11]:
zz, lows, highs = zig_zag(data,3)

In [5]:
parsed_ds=parsed_ds.assign(zz=zz,lows=lows,highs=highs)

In [27]:
ds=data.loc[(data['time']>='2021-03-19 09:00:00')&(data['time']<='2021-03-23 19:00:00')]

In [6]:
fig = go.Figure(data=[go.Candlestick(x=parsed_ds['time'],
                open=parsed_ds['open'],
                high=parsed_ds['high'],
                low=parsed_ds['low'],
                close=parsed_ds['close'])])

fig.add_trace(go.Scatter(
    x=parsed_ds.loc[(parsed_ds['zz'].notnull())]['time'],
    y=parsed_ds.loc[(parsed_ds['zz'].notnull())]['zz']
))


fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
        dict(bounds=[19, 9], pattern="hour"), #hide hours outside of 9am-6pm
    ]
)
fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

In [9]:
def zz_direction(row):
    if row['zz']!=np.nan:
        if row["zz"]==row["highs"] :
            return -1
        else:
            return 1
    return 0

def leg_start(_data):
    leg_start=np.full(_data.shape[0], np.nan)
    leg_counter=0
    for idx in range(_data.shape[0]):
        if not np.isnan(_data.iloc[idx]["zz"]):
            zz_direc=zz_direction(_data.iloc[idx])
            found_start_bar=False
            forward_idx=0
            while not found_start_bar:
                bar_dir = bars.bar_direction(_data.iloc[idx+forward_idx])
                if zz_direc == bar_dir or \
                    0 == bar_dir:
                    leg_start[idx+forward_idx]=leg_counter
                    leg_counter+=1
                    found_start_bar=True
                forward_idx+=1
    return pd.DataFrame(leg_start).ffill()

parsed_ds['leg_start']=leg_start(parsed_ds)
parsed_ds.head(20)

,time,open,high,low,close,zz,lows,highs,leg_start
0,2021-03-19 09:00:00,5530.0,5541.5,5525.5,5533.5,NaN,NaN,NaN,NaN
1,2021-03-19 09:05:00,5533.0,5541.0,5525.5,5527.5,NaN,NaN,NaN,NaN
2,2021-03-19 09:10:00,5527.5,5540.0,5526.0,5538.5,NaN,NaN,NaN,NaN
3,2021-03-19 09:15:00,5538.5,5546.0,5530.5,5545.0,NaN,NaN,NaN,NaN
4,2021-03-19 09:20:00,5545.0,5553.0,5543.0,5548.0,5553.0,NaN,5553.0,NaN
5,2021-03-19 09:25:00,5548.0,5551.5,5543.5,5547.5,NaN,NaN,NaN,0.0
6,2021-03-19 09:30:00,5547.5,5549.5,5540.0,5543.5,NaN,NaN,NaN,0.0
7,2021-03-19 09:35:00,5543.0,5546.5,5533.0,5534.0,NaN,NaN,NaN,0.0
8,2021-03-19 09:40:00,5534.0,5538.0,5529.0,5537.5,NaN,5529.0,NaN,0.0
9,2021-03-19 09:45:00,5537.0,5541.5,5535.5,5539.5,NaN,NaN,NaN,0.0


In [113]:
parsed_ds.loc[parsed_ds['time']=='2021-03-22 12:40:00']

,time,open,high,low,close,zz,highs,lows,leg_start
44,2021-03-22 12:40:00,5538.5,5539.0,5530.0,5532.5,5539.0,5539.0,NaN,1.0


In [42]:
fig = go.Figure(data=[go.Candlestick(x=parsed_ds['time'],
                open=parsed_ds['open'],
                high=parsed_ds['high'],
                low=parsed_ds['low'],
                close=parsed_ds['close'])])

# fig.add_trace(go.Scatter(
#     x=data.loc[(data['zz'].notnull())]['time'],
#     y=data.loc[(data['zz'].notnull())]['zz']
# ))

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
        dict(bounds=[18, 9], pattern="hour"), #hide hours outside of 9am-6pm
    ]
)

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

In [6]:
# Perna total expandida comparada com
# a perna como ela aparece no grafico.
# Relação continua de sobreposição entre 
# as barras.

contracted_leg = parsed_ds.iloc[parsed_ds.shape[0]-1]['high']-parsed_ds.iloc[0]['low']
expanded_leg=0
for idx in range(parsed_ds.shape[0]):
    expanded_leg+=parsed_ds.iloc[idx]['high']-parsed_ds.iloc[idx]['low']

print('expanded_leg: {}, contracted_leg: {}'.format(expanded_leg,contracted_leg))

expanded_leg: 93.0, contracted_leg: 32.5


In [7]:
leg_direction = -1

In [8]:
'''
Bars that trade below 50% of the bar before
'''

overlap_count=0
for idx in range(data.shape[0]):
    if idx <= 1:
        continue
    fifty=data.iloc[idx-1]['low']+(data.iloc[idx-1]['high']-data.iloc[idx-1]['low'])/2
    if leg_direction > 0 and fifty > data.iloc[idx]['low']:
        overlap_count+=1
    if leg_direction < 0 and fifty < data.iloc[idx]['high']:
        overlap_count+=1
    
    # print(idx)


In [18]:
def between(value, bar, body=True):
    direction = bars.bar_direction(bar)
    if body:
        if direction>0:
            if bar['close']>value and bar['open'] < value:
                return True
        if direction<0:
            if bar['close']<value and bar['open'] > value:
                return True
    else:
        if bar['high']>value and bar['low'] < value:
            return True
    return False

def inside_bar(outside, inside, inside_body=False):
    if not inside_body:
        if outside['high']>=inside['high'] and outside['low']<=inside['low']:
            return True
    else:
        direction=bars.bar_direction(inside)
        if direction>0:
            if outside['high']>=inside['close'] and outside['low']<=inside['open']:
                return True
        if direction<0:
            if outside['high']>=inside['open'] and outside['low']<=inside['close']:
                return True
    return False

In [57]:
inside_bar(parsed_ds.iloc[6],parsed_ds.iloc[7]) or \
inside_bar(parsed_ds.iloc[6],parsed_ds.iloc[7], inside_body=True) or \
between(parsed_ds.iloc[6]['close'],parsed_ds.iloc[7]) or \
between(parsed_ds.iloc[6]['open'],parsed_ds.iloc[7])

True

In [63]:
'''
Bars that overlap with the body of bars before
- The first bar of the range does not count
- The body overlaps with two bars before or more
'''

overlaping_count=bar_before=last_overlaping=0
range_starts=[]
while bar_before<parsed_ds.shape[0]-2:
    for overlap_bar in range(bar_before+2, parsed_ds.shape[0]):
        doji=bars.doji(parsed_ds.iloc[bar_before])
        if doji:
            overlap_flag = inside_bar(parsed_ds.iloc[bar_before],parsed_ds.iloc[overlap_bar]) or \
                        inside_bar(parsed_ds.iloc[bar_before],parsed_ds.iloc[overlap_bar], inside_body=True) or \
                        between(parsed_ds.iloc[bar_before]['high'],parsed_ds.iloc[overlap_bar]) or \
                        between(parsed_ds.iloc[bar_before]['low'],parsed_ds.iloc[overlap_bar])
            if overlap_bar == bar_before+2:
                if not overlap_flag:
                    break
            if overlap_flag:
                overlaping_count+=1
                last_overlaping=overlap_bar
                # print('bar_before: ', bar_before)
        if not doji:
            overlap_flag=inside_bar(parsed_ds.iloc[bar_before],parsed_ds.iloc[overlap_bar]) or \
                    inside_bar(parsed_ds.iloc[bar_before],parsed_ds.iloc[overlap_bar], inside_body=True) or \
                    between(parsed_ds.iloc[bar_before]['close'],parsed_ds.iloc[overlap_bar]) or \
                    between(parsed_ds.iloc[bar_before]['open'],parsed_ds.iloc[overlap_bar])
            if overlap_bar == bar_before+2:
                if not overlap_flag:
                    break
            if overlap_flag:
                overlaping_count+=1
                last_overlaping=overlap_bar

    if last_overlaping>bar_before:
        # print('last_overlaping: {} bar_before: {}'.format(last_overlaping,bar_before))
        range_starts.append(bar_before)
        bar_before=last_overlaping
    else:
        bar_before+=1
        # print('bar_before: ', bar_before)

overlaping_count+=len(range_starts)
print('overlaping_count: ', overlaping_count)
print('range_starts: ', range_starts)

last_overlaping: 4 bar_before: 0
last_overlaping: 10 bar_before: 6
overlaping_count:  8
range_starts:  [0, 6]
